**You need to put the files in your drive and mount them**


In [6]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier
data = pd.read_parquet('/content/drive/MyDrive/Class1.parquet', engine='pyarrow')
wdata = pd.read_parquet('/content/drive/MyDrive/Class2.parquet', engine='pyarrow')
validation = pd.read_parquet('/content/drive/MyDrive/validation.parquet', engine='pyarrow')
data =data.T
wdata = wdata.T
validation = validation.T
d = {'value': [0 for i in range(1677)]}
data_y =pd.DataFrame(data=d)
d2 = {'value': [1 for i in range(200)]}
wdata_y =pd.DataFrame(data=d2)
data = torch.tensor(np.array(data))
data_y = torch.tensor(np.array(data_y))
wdata =  torch.tensor(np.array(wdata))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data, data_y, test_size=0.33, random_state=42)

In [4]:
X_train=torch.tensor(np.array(X_train))
X_test = torch.tensor(np.array(X_test))
y_train = torch.tensor(np.array(y_train))
y_test = torch.tensor(np.array(y_test))


In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.sigmoid(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 10
rnn = RNN(1, n_hidden, 1)

In [ ]:
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[1]):
        input = torch.unsqueeze(line_tensor[0][i], 0)
        input = torch.unsqueeze(input, 0)
        output, hidden = rnn(input.float(), hidden)

    return output

In [ ]:
learning_rate = 0.005 
criterion = nn.NLLLoss()
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()
    for i in range(line_tensor.size()[1]):
        input = torch.unsqueeze(line_tensor[0][i], 0)
        input = torch.unsqueeze(input, 0)
        output, hidden = rnn(input.float(), hidden)
    
    loss = criterion(output.float().view((-1)), torch.tensor([1]).float())
    loss.backward()

    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [ ]:
n_iters = 100
criterion = nn.BCELoss()

category_tensor = torch.tensor([0,1])
rnn = RNN(1, n_hidden, 1)
category_tensor = torch.tensor([0,1])

current_loss = 0
all_losses = []

for iter in range(1, n_iters + 1):
    line_tensor = torch.tensor(X_train.sample().values)
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss
  
print(current_loss)


In [ ]:
score=0
for i in np.array(X_test):
  output = evaluate(torch.tensor(i))
  if(output.view((-1)).view((-1)) >0.5):
    score=score+1

**ne converge pas**